In [4]:
import bempp.api as bem
import numpy as np

def function(x, y, i, j):
    zx = 0
    zy = 0
    if i == 0:
        zx = np.sin(x * np.pi * 2)
    if i == 1:
        if x <= 0.25:
            zx = np.sin(x * np.pi * 4)
    if i == 2:
        if x > 0.25:
            zx = -np.sin(x * np.pi * 4)
    if i == 3:
        if x <= 0.5 / 3.:
            zx = np.sin(x * np.pi * 6)
    if i == 4:
        if x > 0.5 / 3. and x <= 1 / 3.:
            zx = -np.sin(x * np.pi * 6)
    if i ==5:
        if x > 1/3.:
            zx = np.sin(x * np.pi * 6)

    if j == 0:
        zy = np.sin(y * np.pi * 2)

    if j == 1:
        if y <= 0.25:
            zy = np.sin(y * np.pi * 4)

    if j == 2:
        if y > 0.25:
            zy = -np.sin(y * np.pi * 4)

    if j == 3:
        if y <= 0.5 / 3.:
            zy = np.sin(y * np.pi * 6)

    if j == 4:
        if y > 0.5 / 3. and y <= 1 / 3.:
            zy = -np.sin(y * np.pi * 6)
    if j == 5:
        if y > 1/3.:
            zy = np.sin(y * np.pi * 6)
    return zx * zy


# Def random:

def Phiz(point, n, domain_index, result):
    x, y, z = point

    res = 0j
    if z == 0.5 and (x <= 0.5) and (y <= 0.5):
        for ii in range(6):
            for jj in range(6):
                res += Random[ii,jj] * function(x, y, ii,jj)	
    result[0] = res



def perturbate(grid, t, kappa_pert=None):
    P1 = bem.function_space(grid, 'P', 1)
    grid_funz = bem.GridFunction(P1, fun = Phiz)
    elements = grid.leaf_view.elements
    vertices = grid.leaf_view.vertices
    normals = P1.global_dof_normals
    x, y, z = vertices
    
    vertices[2, :] = z + t * grid_funz.coefficients
    return bem.grid_from_element_data(vertices, elements)
    
def dirichlet_fun(x, n, domain_index, result):
    result[0] = np.exp(1j * k0 * x[0])

def neumann_fun(x, n, domain_index, result):
    result[0] = 1j * k0 * np.exp(1j * k0 * x[0]) * n[0]



lower_left = [-1.5,-1.5,-1.5]
upper_right = [2.5,2.5,2.5]
file_name = "output.msh"
ndims = [100,100,100]

#pylint: disable=invalid-name
ll = {"xy": (lower_left[0], lower_left[1]),
      "xz": (lower_left[0], lower_left[2]),
      "yz": (lower_left[1], lower_left[2])}
ur = {"xy": (upper_right[0], upper_right[1]),
      "xz": (upper_right[0], upper_right[2]),
      "yz": (upper_right[1], upper_right[2])}
nd = {"xy": (ndims[0], ndims[1]),
      "xz": (ndims[0], ndims[2]),
      "yz": (ndims[1], ndims[2])}


Idx = []
Points = []

for mode in ["xy","yz"]:
    grid = bem.structured_grid(
        ll[mode], ur[mode], nd[mode], axis=mode,offset=0.5)
    nnodes = grid.leaf_view.entity_count(2)
    nelements = grid.leaf_view.entity_count(0)
    space = bem.function_space(
        grid, "P", 1, domains=[0], closed=True)
    points = space.global_dof_interpolation_points
    
    count = np.arange(points.shape[1])
    x,y,z = points
    idx = (x > 1.0) | (x<0) | (y > 1.0) | (y < 0) | (z > 1.0) | (z < 0) 
    
    Idx.append(idx)
    Points.append(points)


k0 = 4
precision = 30

#tolerance = 1E-7

h = 2.0 * np.pi / (precision * k0)

#h = .05

tolerance = 10E-4
grid = bem.shapes.reentrant_cube(h=h)

N = int(grid.leaf_view.entity_count(0))
print(N, 'N')

space = bem.function_space(grid, 'DP', 0)	



theta = np.linspace(0, 2 * np.pi, 400)
points = np.array([np.cos(theta), np.sin(theta), np.zeros(len(theta))])

def solver(grid):
    space = bem.function_space(grid, 'DP', 0)	

    V = bem.operators.boundary.helmholtz.single_layer(space, space, space, k0)
    K = bem.operators.boundary.helmholtz.double_layer(space, space, space, k0)
    I = bem.operators.boundary.sparse.identity(space, space, space)

    df =  bem.GridFunction(space , fun = dirichlet_fun)

    phi, info, res = bem.linalg.gmres(V, df, tol=tolerance, return_residuals=True, use_strong_form=False, maxiter=1000, restart=1000)

    slp_far_field = bem.operators.far_field.helmholtz.single_layer(space, points, k0)

    U = -slp_far_field * phi


    Neo = []
    for i in range(2):
        slp_pot = bem.operators.potential.helmholtz.single_layer(phi.space, Points[i], k0)
        vals = np.exp(1j *k0 * Points[i][0, :]) - slp_pot.evaluate(phi)

        neo = np.zeros(Points[i].shape[1])
        neo[:] = 10000

        neo[Idx[i]] = np.real(np.sum(vals * vals.conj(), axis=0))[Idx[i]]
        Neo.append(neo)

    return U, Neo


U, Neo = solver(grid)


M = 5

t = .075

solt = []
soltX = []
soltY = []
for i in range(M):
    print(i)
    Random = np.random.rand(6,6) * 2  - 1
    print(Random)
    gridt = perturbate(grid, t)
    
    Ut, Neot = solver(gridt)

    solt.append(Ut)
    soltX.append(Neot[0])
    soltY.append(Neot[1])




Umean = np.sum(np.array(solt), axis = 0) / M
UmeanX = np.sum(np.array(soltX), axis = 0) / M
UmeanY = np.sum(np.array(soltY), axis = 0) / M


vart = []
vartX = []
vartY = []

for i in range(M):
    vart.append( (solt[i] - Umean) *np.conj(solt[i] - Umean) )
    vartX.append( (soltX[i] - UmeanX) *np.conj(soltX[i] - UmeanX) )
    vartY.append( (soltY[i] - UmeanY) *np.conj(soltY[i] - UmeanY) )


Uvar = np.sum(np.array(vart), axis=0) / M
UvarX = np.sum(np.array(vartX), axis=0) / M
UvarY = np.sum(np.array(vartY), axis=0) / M

output = [k0, precision, N, M, Umean, UmeanX, UmeanY, Uvar, UvarX, UvarY]


6550 N
0
[[ 0.89643405 -0.66354805 -0.3561693   0.37189978 -0.39544749  0.54247528]
 [ 0.55405557 -0.35570148  0.92777355  0.27844372  0.40276902  0.78247562]
 [ 0.78135703 -0.74901632 -0.20037133  0.60647586  0.86678454  0.41812013]
 [ 0.6164448   0.26097244  0.42460386  0.58599677 -0.06951643 -0.55911596]
 [ 0.32179459 -0.51055511  0.27846488  0.39826047  0.74508885  0.24655697]
 [ 0.46254013 -0.64712763 -0.88975653 -0.11526509  0.58888333  0.42647528]]
1
[[ 0.65677417  0.61201162 -0.11600171 -0.90895697 -0.73099841 -0.86032936]
 [-0.50931117 -0.48000938  0.5990965   0.8964203  -0.63461393  0.50437217]
 [-0.69157849  0.97839024  0.27789733 -0.74419729 -0.87553938  0.30550628]
 [ 0.87414862 -0.41760582  0.9154998  -0.24062377 -0.72993426  0.46503374]
 [ 0.77440901  0.41574695 -0.81851609  0.13795005  0.92372011 -0.65363434]
 [ 0.16680119 -0.74278291 -0.85230172  0.93386798 -0.9089983  -0.94241779]]
2
[[-0.34570602  0.90840545  0.80839248  0.0688699  -0.39448923  0.20392457]
 [-0.57257